In [1]:
from services.question_generator_agent import QuestionGeneratorAgent

In [2]:

agent = QuestionGeneratorAgent.get_question_generator_agent()

messages = {"messages": [{"role":"user", "content": "hey, give me some questions for practicing for my exam"}]}

response = agent.invoke(messages)

In [3]:
print(response['messages'][-1].content)

Here are some practice questions based on sentiment analysis:

1. What does sentiment analysis primarily evaluate in a text document?
   a) The grammatical correctness
   b) The positivity or negativity
   c) The length of the document
   d) The font style used

**Correct answer:** b) The positivity or negativity  
*Explanation:* Sentiment analysis evaluates whether a text is positive, negative, or neutral in tone.

2. Which of the following workloads can benefit from sentiment analysis?
   a) Quantifying product reviews
   b) Prioritizing customer service responses
   c) Translating documents
   d) Detecting spam emails

**Correct answers:** a) Quantifying product reviews, b) Prioritizing customer service responses  
*Explanation:* Sentiment analysis is useful for evaluating reviews and prioritizing responses based on tone. It is not directly related to translation or spam detection.

3. When using Azure AI Language for sentiment evaluation, what information is included in the respons

In [8]:
print(response['structured_response'].response[1])

question='Which of the following workloads can benefit from sentiment analysis?' answers=['Quantifying product reviews', 'Prioritizing customer service responses', 'Translating documents', 'Detecting spam emails'] correct_answer='Quantifying product reviews, Prioritizing customer service responses' explanation='Sentiment analysis is useful for evaluating reviews and prioritizing responses based on tone. It is not directly related to translation or spam detection.'


In [6]:
print(response['structured_response'].answers[0])

B) To evaluate how positive or negative a text document is


In [35]:
print(response['structured_response'].correctness)

AttributeError: 'QuestionAgentResponse' object has no attribute 'correctness'

In [2]:
from services.url_extractor_service import UrlExtractor
from dotenv import load_dotenv
load_dotenv()
import os

api_key = os.getenv('SCRAPPING_TOOL_API_KEY')
url = "https://learn.microsoft.com/en-us/training/courses/ai-102t00"

exam_urls_list = UrlExtractor.scrape_course_modules_units_and_topics(url, api_key)

In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os

client = MongoClient(os.getenv("MONGO_DB_URI"), server_api=ServerApi('1'))

In [8]:
db = client['documentation']
collection = db["AI-900"]

docs = collection.find()

for doc in docs:
    print(doc['documents'])

[{'source': 'microsoft.com', 'name': 'AI-900 Overview'}, {'source': 'microsoft.com', 'name': 'AI-900 Chapter 1'}]


In [6]:
import json

exam_urls = json.loads(exam_urls_list)

In [95]:
topic_url = exam_urls['modules'][2]['units'][0]['topics'][4]['url']
topic_url

'https://learn.microsoft.com/en-us/training/modules/analyze-text-ai-language/5-analyze-sentiment'

In [96]:
name = exam_urls['modules'][2]['units'][0]['topics'][4]['name']
name

'Analyze sentiment'

In [39]:

from langchain_community.document_loaders import ScrapingAntLoader

scrapingant_loader = ScrapingAntLoader(
    [topic_url],  # List of URLs to scrape
    api_key=api_key,  # Get your API key from https://scrapingant.com/
    continue_on_failure=True,  # Ignore unprocessable web pages and log their exceptions
)

In [42]:
docs = scrapingant_loader.load()

In [ ]:
full_text = docs[0].page_content


In [68]:
filtered_text = full_text.partition(f"# {name}")

In [ ]:
context = filtered_text[2].partition("## Next unit:")[0]

In [ ]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
from langchain_core.tools import tool
load_dotenv()
import os


azure_openai_endpoint = "https://par-mcdv1i6v-eastus2.cognitiveservices.azure.com/"
azure_openai_key = os.getenv('AZURE_OPENAI_KEY')
deployment_name = "gpt-4.1-nano"
llm = AzureChatOpenAI(azure_endpoint=azure_openai_endpoint, 
                      api_key=azure_openai_key, 
                      api_version="2024-12-01-preview",
                      azure_deployment=deployment_name
                      )

In [90]:
prompt = f"""You're a smart assistant that helps users to prepare themselves to pass Microsoft certification exams.
You job is to generate practice questions for the user.

Use the following context to generate your sample questions.

### Context ###
{context}

Generate the questions and answers, each question have to have 3 answers, one correct and 2 incorrect, Or all of them correct, you decide.

Provide also the explanation for each answer and the source (if available).

Generate only 3 sample questions.

Level of difficulty of questions: basic.

"""

In [91]:

response = llm.invoke(prompt).content

print(response)

**Question 1:**  
What does sentiment analysis evaluate in a text document?  

A) The grammatical correctness of the text.  
B) Whether the text is positive, negative, or neutral. *(Correct)*  
C) The readability level of the text.  

**Explanation:**  
Sentiment analysis assesses the overall emotional tone or attitude expressed in a text, categorizing it as positive, negative, or neutral. It does not evaluate grammar or readability.  

**Source:**  
Based on the context provided, which describes how sentiment analysis evaluates positive or negative sentiments in text.

---

**Question 2:**  
In Azure AI Language sentiment analysis, what determines the overall sentiment of a document?  

A) The sentiment of the first sentence only.  
B) The sentiment classification of individual sentences and their combination. *(Correct)*  
C) The length of the document.  

**Explanation:**  
The overall document sentiment is derived from the classification of individual sentences: if all are neutral,